In [279]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [280]:
housing_data_df = pd.read_csv('AmesHousing.tsv', sep='\t')

In [281]:
housing_data_df.head()

,Order,PID,MS SubClass,MS Zoning,Lot Frontage,Lot Area,Street,Alley,Lot Shape,Land Contour,...,Pool Area,Pool QC,Fence,Misc Feature,Misc Val,Mo Sold,Yr Sold,Sale Type,Sale Condition,SalePrice
0,1,526301100,20,RL,141.0,31770,Pave,NaN,IR1,Lvl,...,0,NaN,NaN,NaN,0,5,2010,WD,Normal,215000
1,2,526350040,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,...,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal,105000
2,3,526351010,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,...,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal,172000
3,4,526353030,20,RL,93.0,11160,Pave,NaN,Reg,Lvl,...,0,NaN,NaN,NaN,0,4,2010,WD,Normal,244000
4,5,527105010,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,...,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal,189900


In [282]:
housing_data_df.shape

(2930, 82)

Lets use 80% of the data as train set, and the rest 20% as test

In [283]:
def get_train_and_test_sets(data, split=0.8):
    df = data.sample(frac=1)
    train_index = int(df.shape[0]*0.8)
    train = df[:train_index]
    test = df[train_index:]
    return (train, test)

In [284]:
train, test = get_train_and_test_sets(housing_data_df)

# Predicting models

Let's establish a minimum quality by estimating accuracy of prediction by just one column: "Gr Liv Area"

In [285]:
lr1 = LinearRegression()
lr1.fit(train[['Gr Liv Area']], train['SalePrice'])

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [286]:
lr1_train_pred = lr1.predict(train[['Gr Liv Area']])
lr1_train_rmse = mean_squared_error(lr1_train_pred, train['SalePrice']) ** 0.5

lr1_test_pred = lr1.predict(test[['Gr Liv Area']])
lr1_test_rmse = mean_squared_error(lr1_test_pred, test['SalePrice']) ** 0.5

print(lr1_train_rmse, lr1_test_rmse)

56509.1103911 56538.5852969


Let's check the results by ourselves

In [287]:
def compare_predictions(predicted, actual):
    check_df = pd.DataFrame(data=predicted, index=actual.index, columns=["Predicted SalePrice"])
    check_df['SalePrice'] = actual
    check_df["Error, %"] = np.abs(check_df["Predicted SalePrice"]*100/check_df['SalePrice'] - 100)
    return (check_df.sort_index(), check_df["Error, %"].mean())

In [288]:
check_df, avg_error = compare_predictions(lr1_test_pred, test['SalePrice'])

In [289]:
check_df.head(10)

,Predicted SalePrice,SalePrice,"Error, %"
8,193932.628554,236500,17.998889
15,383185.057363,538000,28.776012
18,108353.658942,141000,23.153433
26,110402.091726,126000,12.379292
27,108353.658942,115000,5.779427
31,134300.474197,88000,52.614175
35,175724.337147,146000,20.359135
36,275642.336242,376162,26.722440
41,203264.377900,275000,26.085681
44,279056.390881,611657,54.376981


In [290]:
avg_error

23.013630955190397

### For the simplest model the average error is ~25%

# Editing features for initial dataset

In [291]:
feature_edited_df = housing_data_df.copy()

In [292]:
null_series = feature_edited_df.isnull().sum()
null_series = null_series[null_series > 0]
null_series

Lot Frontage       490
Alley             2732
Mas Vnr Type        23
Mas Vnr Area        23
Bsmt Qual           80
Bsmt Cond           80
Bsmt Exposure       83
BsmtFin Type 1      80
BsmtFin SF 1         1
BsmtFin Type 2      81
BsmtFin SF 2         1
Bsmt Unf SF          1
Total Bsmt SF        1
Electrical           1
Bsmt Full Bath       2
Bsmt Half Bath       2
Fireplace Qu      1422
Garage Type        157
Garage Yr Blt      159
Garage Finish      159
Garage Cars          1
Garage Area          1
Garage Qual        159
Garage Cond        159
Pool QC           2917
Fence             2358
Misc Feature      2824
dtype: int64

Lets remove entirely columns that have more than 15% of missing data, and lets fill the rest with average data

In [293]:
removal_threshold = int(feature_edited_df.shape[0]*0.15)

In [294]:
null_series_to_drop = null_series[null_series > removal_threshold]

In [295]:
null_series_to_fill = null_series[null_series <= removal_threshold]
null_series_to_fill = feature_edited_df[null_series_to_fill.index].select_dtypes(include=['float'])

In [296]:
for col in null_series_to_drop.index:
    del feature_edited_df[col]

In [297]:
feature_edited_df[null_series_to_fill.columns] = null_series_to_fill.fillna(null_series_to_fill.mean())

In [298]:
null_series = feature_edited_df.isnull().sum()
null_series = null_series[null_series > 0]
null_series

Mas Vnr Type       23
Bsmt Qual          80
Bsmt Cond          80
Bsmt Exposure      83
BsmtFin Type 1     80
BsmtFin Type 2     81
Electrical          1
Garage Type       157
Garage Finish     159
Garage Qual       159
Garage Cond       159
dtype: int64

In [299]:
feature_edited_df[null_series.index].head()

,Mas Vnr Type,Bsmt Qual,Bsmt Cond,Bsmt Exposure,BsmtFin Type 1,BsmtFin Type 2,Electrical,Garage Type,Garage Finish,Garage Qual,Garage Cond
0,Stone,TA,Gd,Gd,BLQ,Unf,SBrkr,Attchd,Fin,TA,TA
1,None,TA,TA,No,Rec,LwQ,SBrkr,Attchd,Unf,TA,TA
2,BrkFace,TA,TA,No,ALQ,Unf,SBrkr,Attchd,Unf,TA,TA
3,None,TA,TA,No,ALQ,Unf,SBrkr,Attchd,Fin,TA,TA
4,None,Gd,TA,No,GLQ,Unf,SBrkr,Attchd,Fin,TA,TA


Now only categorical data has some missing values. Lets transform it first and then decide what to do.

In [300]:
categorical_missing_data = null_series

In [301]:
text_cols = feature_edited_df.select_dtypes(include=['object']).columns
print(text_cols)

Index(['MS Zoning', 'Street', 'Lot Shape', 'Land Contour', 'Utilities',
       'Lot Config', 'Land Slope', 'Neighborhood', 'Condition 1',
       'Condition 2', 'Bldg Type', 'House Style', 'Roof Style', 'Roof Matl',
       'Exterior 1st', 'Exterior 2nd', 'Mas Vnr Type', 'Exter Qual',
       'Exter Cond', 'Foundation', 'Bsmt Qual', 'Bsmt Cond', 'Bsmt Exposure',
       'BsmtFin Type 1', 'BsmtFin Type 2', 'Heating', 'Heating QC',
       'Central Air', 'Electrical', 'Kitchen Qual', 'Functional',
       'Garage Type', 'Garage Finish', 'Garage Qual', 'Garage Cond',
       'Paved Drive', 'Sale Type', 'Sale Condition'],
      dtype='object')


In [302]:
for col in text_cols:
    feature_edited_df[col] = feature_edited_df[col].astype('category')
    col_dummies = pd.get_dummies(feature_edited_df[col], prefix=col)
    feature_edited_df = pd.concat([feature_edited_df, col_dummies], axis=1)
    del feature_edited_df[col]

All missing categorical values have been replaced with zeroes in all related one-hot-encoding columns

In [303]:
null_series = feature_edited_df.isnull().sum()
null_series = null_series[null_series > 0]
null_series

Series([], dtype: int64)

No missing values so far

In [312]:
train, test = get_train_and_test_sets(feature_edited_df)

In [313]:
def get_features(dataset, target='SalePrice'):
    features = []
    for col in dataset.columns:
        if col != target:
            features.append(col)
    return features

In [314]:
features = get_features(feature_edited_df)

## Test new model:

In [315]:
lr2 = LinearRegression()
lr2.fit(train[features], train['SalePrice'])

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [316]:
lr2_train_pred = lr2.predict(train[features])
lr2_train_rmse = mean_squared_error(lr2_train_pred, train['SalePrice']) ** 0.5

lr2_test_pred = lr2.predict(test[features])
lr2_test_rmse = mean_squared_error(lr2_test_pred, test['SalePrice']) ** 0.5

print(lr2_train_rmse, lr2_test_rmse)

21088.5432053 39246.2550554


In [317]:
check_df, avg_error = compare_predictions(lr2_test_pred, test['SalePrice'])

In [318]:
avg_error

9.999680286305194

In [319]:
check_df.head(10)

,Predicted SalePrice,SalePrice,"Error, %"
9,194397.336329,189000,2.855734
13,204602.533510,171500,19.301769
15,469381.764198,538000,12.754319
22,217216.630226,216000,0.563255
27,95088.621075,115000,17.314243
28,178980.276404,184000,2.728111
30,89202.019042,105500,15.448323
33,146179.140627,149900,2.482228
38,399398.207127,395192,1.064345
40,252374.169757,220000,14.715532


# Average error got down to ~10%

Now lets improve data on numerical values that are not very representative.

In [320]:
numerical_data = housing_data_df.select_dtypes(include=['float32', 'int64'])

In [321]:
numerical_data.columns

Index(['Order', 'PID', 'MS SubClass', 'Lot Area', 'Overall Qual',
       'Overall Cond', 'Year Built', 'Year Remod/Add', '1st Flr SF',
       '2nd Flr SF', 'Low Qual Fin SF', 'Gr Liv Area', 'Full Bath',
       'Half Bath', 'Bedroom AbvGr', 'Kitchen AbvGr', 'TotRms AbvGrd',
       'Fireplaces', 'Wood Deck SF', 'Open Porch SF', 'Enclosed Porch',
       '3Ssn Porch', 'Screen Porch', 'Pool Area', 'Misc Val', 'Mo Sold',
       'Yr Sold', 'SalePrice'],
      dtype='object')

We don't need columns "Order" and "PID", and we also have to modify all columns related to years.

Let's remove column "Year Built", and replace 'Year Remod/Add', 'Garage Yr Blt', 'Yr Sold' with difference with the value and "Year Built".

In [322]:
feature_edited_2_df = feature_edited_df.copy()

In [323]:
del feature_edited_2_df['Order']
del feature_edited_2_df['PID']

In [324]:
for col in ['Year Remod/Add', 'Garage Yr Blt', 'Yr Sold']:
    feature_edited_2_df[col] = feature_edited_2_df[col] - feature_edited_2_df['Year Built']
    
del feature_edited_2_df['Year Built']

Also let's transform 'MS SubClass' into categorical type. It is reresented as numerical, but numbers represent category anyway, so it is sort of misleading

In [325]:
feature_edited_2_df['MS SubClass'] = feature_edited_2_df['MS SubClass'].astype('category')
col_dummies = pd.get_dummies(feature_edited_2_df['MS SubClass'])
feature_edited_2_df = pd.concat([feature_edited_2_df, col_dummies], axis=1)
del feature_edited_2_df['MS SubClass']

In [326]:
def test_model(train, test, features):    
    lr = LinearRegression()
    lr.fit(train[features], train['SalePrice'])
    
    lr_train_pred = lr.predict(train[features])
    lr_train_rmse = mean_squared_error(lr_train_pred, train['SalePrice']) ** 0.5

    lr_test_pred = lr.predict(test[features])
    lr_test_rmse = mean_squared_error(lr_test_pred, test['SalePrice']) ** 0.5

    print("RMSE TRAIN AND TEST:")
    print(lr_train_rmse, lr_test_rmse)
    
    check_df, avg_error = compare_predictions(lr_test_pred, test['SalePrice'])
    
    print("AVERAGE ERROR:", avg_error)
    
    print(check_df.head(10))
    
    return lr

In [327]:
train, test = get_train_and_test_sets(feature_edited_2_df)
features = get_features(feature_edited_2_df)

In [328]:
lr3 = test_model(train, test, features)

RMSE TRAIN AND TEST:
22271.5566443 23648.317037
AVERAGE ERROR: 9.67346379322944
    Predicted SalePrice  SalePrice   Error, %
4         186189.262792     189900   1.954048
7         209056.896644     191500   9.168092
11        176795.821398     185000   4.434691
27         88606.741110     115000  22.950660
35        138535.275495     146000   5.112825
45        222063.367137     224000   0.864568
48        315413.445727     319900   1.402486
50        176431.996390     175500   0.531052
51        174340.457022     199500  12.611300
62        349658.890804     325000   7.587351


# Average error stayed at 10%

Now lets explore correlation and try to remove all the noise from data

In [329]:
correlated_df = feature_edited_2_df.copy()

In [330]:
correlations = correlated_df.corr()['SalePrice'].abs().sort_values(ascending=False)

Lets assume that correlation below 0.15 is dismissable

In [331]:
correlations = correlations[correlations > 0.15]

In [332]:
correlations

SalePrice               1.000000
Overall Qual            0.799262
Gr Liv Area             0.706780
Garage Cars             0.647861
Garage Area             0.640385
Total Bsmt SF           0.632105
1st Flr SF              0.621676
Bsmt Qual_Ex            0.593567
Exter Qual_TA           0.590897
Yr Sold                 0.558907
Full Bath               0.545604
Kitchen Qual_Ex         0.537561
Kitchen Qual_TA         0.526721
Foundation_PConc        0.520966
Mas Vnr Area            0.505784
TotRms AbvGrd           0.495474
Exter Qual_Ex           0.480477
Fireplaces              0.474558
BsmtFin Type 1_GLQ      0.455326
Heating QC_Ex           0.454553
Bsmt Qual_TA            0.452487
Exter Qual_Gd           0.446489
Neighborhood_NridgHt    0.433299
BsmtFin SF 1            0.432794
Garage Finish_Fin       0.424902
Garage Finish_Unf       0.421876
Mas Vnr Type_None       0.403962
Garage Type_Detchd      0.365074
60                      0.362327
Bsmt Exposure_Gd        0.355665
          

In [333]:
import seaborn as sns
import matplotlib.pyplot as plt

%matplotlib inline

In [334]:
correlated_df = correlated_df[correlations.index]

In [335]:
correlated_df.head()

,SalePrice,Overall Qual,Gr Liv Area,Garage Cars,Garage Area,Total Bsmt SF,1st Flr SF,Bsmt Qual_Ex,Exter Qual_TA,Yr Sold,...,50,Neighborhood_IDOTRR,Exterior 1st_Wd Sdng,Garage Finish_RFn,Exterior 2nd_Wd Sdng,Neighborhood_Edwards,Garage Qual_Fa,Neighborhood_Somerst,BsmtFin Type 1_Rec,Bsmt Qual_Fa
0,215000,6,1656,2.0,528.0,1080.0,1656,0,1,50,...,0,0,0,0,0,0,0,0,0,0
1,105000,5,896,1.0,730.0,882.0,896,0,1,49,...,0,0,0,0,0,0,0,0,1,0
2,172000,6,1329,1.0,312.0,1329.0,1329,0,1,52,...,0,0,1,0,1,0,0,0,0,0
3,244000,7,2110,2.0,522.0,2110.0,2110,0,0,42,...,0,0,0,0,0,0,0,0,0,0
4,189900,5,1629,2.0,482.0,928.0,928,0,1,13,...,0,0,0,0,0,0,0,0,0,0


Lets test with strongly correlated data

In [344]:
train, test = get_train_and_test_sets(correlated_df)
features = get_features(correlated_df)

In [345]:
lr4 = test_model(train, test, features)

RMSE TRAIN AND TEST:
28176.2166065 27828.4029427
AVERAGE ERROR: 10.139621149649104
    Predicted SalePrice  SalePrice   Error, %
3         259266.592530     244000   6.256800
15        449281.843474     538000  16.490364
20        190177.394490     190000   0.093366
26        117505.712908     126000   6.741498
38        388446.073868     395192   1.707000
42        308242.270090     259000  19.012459
48        336625.273738     319900   5.228282
62        348626.676909     325000   7.269747
63        304575.792555     290000   5.026135
68        204446.160048     215200   4.997138


# Average error remained at ~10%

But thank to the latest improvements the solution should've become more reliable